In [125]:
from cps_data_agg import (process_cps_data)
file_path = r'C:\Users\Biswajit Palit\Downloads\cps_00006.csv.gz'
data = process_cps_data(file_path)

data

,STATEFIP,YEAR,Residuals,STATEFIP_2,STATEFIP_4,STATEFIP_5,STATEFIP_6,STATEFIP_8,STATEFIP_9,STATEFIP_10,...,YEAR_1991,YEAR_1992,YEAR_1993,YEAR_1994,YEAR_1995,YEAR_1996,YEAR_1997,YEAR_1998,YEAR_1999,YEAR_2000
0,1,1980,-0.782133,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1981,-0.472124,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1982,-0.589845,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1983,-0.519442,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1984,-0.429119,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,56,1996,0.013756,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1046,56,1997,-0.012077,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1047,56,1998,0.066763,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1048,56,1999,0.122135,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [126]:
data.columns

Index(['STATEFIP', 'YEAR', 'Residuals', 'STATEFIP_2', 'STATEFIP_4',
       'STATEFIP_5', 'STATEFIP_6', 'STATEFIP_8', 'STATEFIP_9', 'STATEFIP_10',
       'STATEFIP_12', 'STATEFIP_13', 'STATEFIP_15', 'STATEFIP_16',
       'STATEFIP_17', 'STATEFIP_18', 'STATEFIP_19', 'STATEFIP_20',
       'STATEFIP_21', 'STATEFIP_22', 'STATEFIP_23', 'STATEFIP_24',
       'STATEFIP_25', 'STATEFIP_26', 'STATEFIP_27', 'STATEFIP_28',
       'STATEFIP_29', 'STATEFIP_30', 'STATEFIP_31', 'STATEFIP_32',
       'STATEFIP_33', 'STATEFIP_34', 'STATEFIP_35', 'STATEFIP_36',
       'STATEFIP_37', 'STATEFIP_38', 'STATEFIP_39', 'STATEFIP_40',
       'STATEFIP_41', 'STATEFIP_42', 'STATEFIP_44', 'STATEFIP_45',
       'STATEFIP_46', 'STATEFIP_47', 'STATEFIP_48', 'STATEFIP_49',
       'STATEFIP_50', 'STATEFIP_51', 'STATEFIP_53', 'STATEFIP_54',
       'STATEFIP_55', 'STATEFIP_56', 'YEAR_1981', 'YEAR_1982', 'YEAR_1983',
       'YEAR_1984', 'YEAR_1985', 'YEAR_1986', 'YEAR_1987', 'YEAR_1988',
       'YEAR_1989', 'YEAR_1990', 'YE

In [127]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.stattools import durbin_watson
from statsmodels.regression import yule_walker

In [131]:
from cps_data_agg import (process_cps_data)
file_path = r'C:\Users\sneha\Downloads\cps_00006.csv.gz'
data = process_cps_data(file_path)

import pandas as pd
import statsmodels.api as sm
import numpy as np

# Assuming your data is loaded into a DataFrame named data
# data = pd.read_csv('path_to_your_data.csv')
states = data['STATEFIP'].unique()
    

    # Randomly select half of the states to be in the treatment group
    
treatment_states = np.random.choice(states, size=len(states)//2, replace=False)

    # Assign treatment year to each treatment state, staggered between 1985 and 1995
treatment_years = np.random.choice(range(1985, 1995), size=len(treatment_states), replace=True)
state_to_treatment_year = dict(zip(treatment_states, treatment_years))

    # Add a treatment column to the DataFrame
data['TREATMENT'] = data.apply(lambda x: 1 if x['STATEFIP'] in treatment_states and x['YEAR'] >= state_to_treatment_year[x['STATEFIP']] else 0, axis=1)


data = data.set_index(['STATEFIP', 'YEAR'])

# Construct the formula for OLS regression
formula = ('Residuals ~ STATEFIP_2 + STATEFIP_4 + STATEFIP_5 + STATEFIP_6 + STATEFIP_8 + '
           'STATEFIP_9 + STATEFIP_10 + STATEFIP_12 + STATEFIP_13 + STATEFIP_15 + '
           'STATEFIP_16 + STATEFIP_17 + STATEFIP_18 + STATEFIP_19 + STATEFIP_20 + '
           'STATEFIP_21 + STATEFIP_22 + STATEFIP_23 + STATEFIP_24 + STATEFIP_25 + '
           'STATEFIP_26 + STATEFIP_27 + STATEFIP_28 + STATEFIP_29 + STATEFIP_30 + '
           'STATEFIP_31 + STATEFIP_32 + STATEFIP_33 + STATEFIP_34 + STATEFIP_35 + '
           'STATEFIP_36 + STATEFIP_37 + STATEFIP_38 + STATEFIP_39 + STATEFIP_40 + '
           'STATEFIP_41 + STATEFIP_42 + STATEFIP_44 + STATEFIP_45 + STATEFIP_46 + '
           'STATEFIP_47 + STATEFIP_48 + STATEFIP_49 + STATEFIP_50 + STATEFIP_51 + '
           'STATEFIP_53 + STATEFIP_54 + STATEFIP_55 + STATEFIP_56 + '
           'YEAR_1981 + YEAR_1982 + YEAR_1983 + YEAR_1984 + YEAR_1985 + '
           'YEAR_1986 + YEAR_1987 + YEAR_1988 + YEAR_1989 + YEAR_1990 + '
           'YEAR_1991 + YEAR_1992 + YEAR_1993 + YEAR_1994 + YEAR_1995 + '
           'YEAR_1996 + YEAR_1997 + YEAR_1998 + YEAR_1999 + YEAR_2000 + TREATMENT')

# Initial OLS Regression
mod_ols = sm.OLS.from_formula(formula, data=data)
model = mod_ols.fit()
print("OLS Results:")
print(model.summary())
ols_resid = model.resid

res_fit = sm.OLS(list(ols_resid[1:]), list(ols_resid[:-1])).fit()
rho = res_fit.params

X = data[['TREATMENT','STATEFIP_2', 'STATEFIP_4',
       'STATEFIP_5', 'STATEFIP_6', 'STATEFIP_8', 'STATEFIP_9', 'STATEFIP_10',
       'STATEFIP_12', 'STATEFIP_13', 'STATEFIP_15', 'STATEFIP_16',
       'STATEFIP_17', 'STATEFIP_18', 'STATEFIP_19', 'STATEFIP_20',
       'STATEFIP_21', 'STATEFIP_22', 'STATEFIP_23', 'STATEFIP_24',
       'STATEFIP_25', 'STATEFIP_26', 'STATEFIP_27', 'STATEFIP_28',
       'STATEFIP_29', 'STATEFIP_30', 'STATEFIP_31', 'STATEFIP_32',
       'STATEFIP_33', 'STATEFIP_34', 'STATEFIP_35', 'STATEFIP_36',
       'STATEFIP_37', 'STATEFIP_38', 'STATEFIP_39', 'STATEFIP_40',
       'STATEFIP_41', 'STATEFIP_42', 'STATEFIP_44', 'STATEFIP_45',
       'STATEFIP_46', 'STATEFIP_47', 'STATEFIP_48', 'STATEFIP_49',
       'STATEFIP_50', 'STATEFIP_51', 'STATEFIP_53', 'STATEFIP_54',
       'STATEFIP_55', 'STATEFIP_56', 'YEAR_1981', 'YEAR_1982', 'YEAR_1983',
       'YEAR_1984', 'YEAR_1985', 'YEAR_1986', 'YEAR_1987', 'YEAR_1988',
       'YEAR_1989', 'YEAR_1990', 'YEAR_1991', 'YEAR_1992', 'YEAR_1993',
       'YEAR_1994', 'YEAR_1995', 'YEAR_1996', 'YEAR_1997', 'YEAR_1998',
       'YEAR_1999', 'YEAR_2000' ]] # plus any other control variables
X = sm.add_constant(X)
Y = data['Residuals'] #

from scipy.linalg import toeplitz
order = toeplitz(np.arange(1050))
sigma = rho**order

gls_model = sm.GLS(Y, X, sigma=sigma)
gls_results = gls_model.fit()
print(gls_results.summary())




OLS Results:
                            OLS Regression Results                            
Dep. Variable:              Residuals   R-squared:                       0.958
Model:                            OLS   Adj. R-squared:                  0.955
Method:                 Least Squares   F-statistic:                     319.1
Date:                Wed, 27 Dec 2023   Prob (F-statistic):               0.00
Time:                        08:07:19   Log-Likelihood:                 1212.7
No. Observations:                1050   AIC:                            -2283.
Df Residuals:                     979   BIC:                            -1932.
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.7608      0.021    

In [129]:
gls_results['TREATMENT'].params()

TypeError: 'RegressionResultsWrapper' object is not subscriptable

In [130]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# Assuming your data is loaded into a DataFrame named data
# data = pd.read_csv('path_to_your_data.csv')

import pandas as pd
import statsmodels.api as sm
import numpy as np


# Assuming your data is loaded into a DataFrame named data
# data = pd.read_csv('path_to_your_data.csv')
states = data['STATEFIP'].unique()
    

    # Randomly select half of the states to be in the treatment group
    
treatment_states = np.random.choice(states, size=len(states)//2, replace=False)

    # Assign treatment year to each treatment state, staggered between 1985 and 1995
treatment_years = np.random.choice(range(1985, 1995), size=len(treatment_states), replace=True)
state_to_treatment_year = dict(zip(treatment_states, treatment_years))

    # Add a treatment column to the DataFrame
data['TREATMENT'] = data.apply(lambda x: 1 if x['STATEFIP'] in treatment_states and x['YEAR'] >= state_to_treatment_year[x['STATEFIP']] else 0, axis=1)


data = data.set_index(['STATEFIP', 'YEAR'])

# Construct the formula for OLS regression
formula = ('Residuals ~ STATEFIP_2 + STATEFIP_4 + STATEFIP_5 + STATEFIP_6 + STATEFIP_8 + '
           'STATEFIP_9 + STATEFIP_10 + STATEFIP_12 + STATEFIP_13 + STATEFIP_15 + '
           'STATEFIP_16 + STATEFIP_17 + STATEFIP_18 + STATEFIP_19 + STATEFIP_20 + '
           'STATEFIP_21 + STATEFIP_22 + STATEFIP_23 + STATEFIP_24 + STATEFIP_25 + '
           'STATEFIP_26 + STATEFIP_27 + STATEFIP_28 + STATEFIP_29 + STATEFIP_30 + '
           'STATEFIP_31 + STATEFIP_32 + STATEFIP_33 + STATEFIP_34 + STATEFIP_35 + '
           'STATEFIP_36 + STATEFIP_37 + STATEFIP_38 + STATEFIP_39 + STATEFIP_40 + '
           'STATEFIP_41 + STATEFIP_42 + STATEFIP_44 + STATEFIP_45 + STATEFIP_46 + '
           'STATEFIP_47 + STATEFIP_48 + STATEFIP_49 + STATEFIP_50 + STATEFIP_51 + '
           'STATEFIP_53 + STATEFIP_54 + STATEFIP_55 + STATEFIP_56 + '
           'YEAR_1981 + YEAR_1982 + YEAR_1983 + YEAR_1984 + YEAR_1985 + '
           'YEAR_1986 + YEAR_1987 + YEAR_1988 + YEAR_1989 + YEAR_1990 + '
           'YEAR_1991 + YEAR_1992 + YEAR_1993 + YEAR_1994 + YEAR_1995 + '
           'YEAR_1996 + YEAR_1997 + YEAR_1998 + YEAR_1999 + YEAR_2000 + TREATMENT')

# Initial OLS Regression
mod_ols = sm.OLS.from_formula(formula, data=data)
model = mod_ols.fit()
print("OLS Results:")
print(model.summary())
ols_resid = model.resid

# Assuming your data is loaded into a DataFrame named data
# data = pd.read_csv('path_to_your_data.csv')

# [Your code for creating treatment groups]

data = data.set_index(['STATEFIP', 'YEAR'])

# [Your code for the OLS regression and extracting residuals]

# Estimate rho using a more robust method if possible
# The following is a simple approach
res_fit = sm.OLS(ols_resid[1:], sm.add_constant(ols_resid[:-1])).fit()
rho = res_fit.params[1]

# [Rest of your code to define X and Y]

# Efficiently create the covariance matrix
n = len(data)
sigma = model.scale
toeplitz_element = [rho**i for i in range(n)]
cov_matrix = sigma * toeplitz(toeplitz_element)

# GLS Model
gls_model = sm.GLS(Y, X, sigma=cov_matrix)
gls_results = gls_model.fit()
print(gls_results.summary())
import pandas as pd
import statsmodels.api as sm
import numpy as np

# Assuming your data is loaded into a DataFrame named data
# data = pd.read_csv('path_to_your_data.csv')

# [Your code for creating treatment groups]

data = data.set_index(['STATEFIP', 'YEAR'])

# [Your code for the OLS regression and extracting residuals]

# Estimate rho using a more robust method if possible
# The following is a simple approach
res_fit = sm.OLS(ols_resid[1:], sm.add_constant(ols_resid[:-1])).fit()
rho = res_fit.params[1]

# [Rest of your code to define X and Y]

# Efficiently create the covariance matrix
n = len(data)
sigma = model.scale
toeplitz_element = [rho**i for i in range(n)]
cov_matrix = sigma * toeplitz(toeplitz_element)

# GLS Model
gls_model = sm.GLS(Y, X, sigma=cov_matrix)
gls_results = gls_model.fit()
print(gls_results.summary())


data = data.set_index(['STATEFIP', 'YEAR'])

# [Your code for the OLS regression and extracting residuals]

# Estimate rho using a more robust method if possible
# The following is a simple approach
res_fit = sm.OLS(ols_resid[1:], sm.add_constant(ols_resid[:-1])).fit()
rho = res_fit.params[1]

# [Rest of your code to define X and Y]

# Efficiently create the covariance matrix
n = len(data)
sigma = model.scale
toeplitz_element = [rho**i for i in range(n)]
cov_matrix = sigma * toeplitz(toeplitz_element)

# GLS Model
gls_model = sm.GLS(Y, X, sigma=cov_matrix)
gls_results = gls_model.fit()
print(gls_results.summary())


KeyError: 'STATEFIP'